In [ ]:
!apt-get install -y coinor-cbc
!pip install pyomo


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5 coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 22 not upgraded.
Need to get 2,908 kB of archives.
After this operation, 8,310 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcoinutils3v5 amd64 2.11.4+repack1-2 [465 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libosi1v5 amd64 0.108.6+repack1-2 [275 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libclp1 amd64 1.17.5+repack1-1 [937 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcgl1 amd64 0.60.3+repack1-3 [420 kB]
Get:5 http:/

In [ ]:
!apt-get install -y coinor-cbc


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
coinor-cbc is already the newest version (2.10.7+ds1-1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:

from pyomo.environ import *

# initialize the model
model = ConcreteModel()

# months 1 to 6
months = range(1, 7)


# 1 is Red and 2 is Black
colors = [1, 2]



# parameters
production_cost = 8

# Red is $24 and black is $18
selling_price = {1: 24, 2: 18}
demand = {
    1: {1: 250, 2: 200},
    2: {1: 90, 2: 160},
    3: {1: 250, 2: 450},
    4: {1: 350, 2: 300},
    5: {1: 320, 2: 200},
    6: {1: 380, 2: 200},
}
inventory_cost = 2
extra_cost_multiplier = .25

# DVs
model.x = Var(months, colors, within=NonNegativeIntegers)  # Shirts produced
model.y = Var(months, colors, within=NonNegativeIntegers)  # Shirts sold
model.e = Var(months, colors, within=NonNegativeIntegers)  # Inventory
model.z = Var(months, within=NonNegativeIntegers)  # Extra production beyond 500

# objective function is to Maximize profits
def objective_rule(model):
    revenue = sum(selling_price[c] * model.y[m, c] for m in months for c in colors)
    production_costs = sum(production_cost * model.x[m, c] for m in months for c in colors)
    extra_costs = sum(extra_cost_multiplier * production_cost * model.z[m] for m in months)
    inventory_costs = sum(inventory_cost * model.e[m, c] for m in months for c in colors)
    return revenue - production_costs - extra_costs - inventory_costs

model.objective = Objective(rule=objective_rule, sense=maximize)

# constraints




# production limit per month
def production_limit_rule(model, m):
    return sum(model.x[m, c] for c in colors) <= 500 + model.z[m]
model.production_limit = Constraint(months, rule=production_limit_rule)

# demand constraint for shirts sold cannot go past demand
def demand_constraint_rule(model, m, c):
    return model.y[m, c] <= demand[m][c]
model.demand_constraint = Constraint(months, colors, rule=demand_constraint_rule)

# inventory flow constraint
def inventory_flow_rule(model, m, c):
    if m == 1:
        return model.x[m, c] - model.y[m, c] == model.e[m, c]
    else:
        return model.e[m-1, c] + model.x[m, c] - model.y[m, c] == model.e[m, c]
model.inventory_flow = Constraint(months, colors, rule=inventory_flow_rule)

# no negativity
def extra_production_rule(model, m):
    return model.z[m] >= 0
model.extra_production = Constraint(months, rule=extra_production_rule)

# solve the model using cbc
solver = SolverFactory('cbc')
result = solver.solve(model)

# Display results
print("Objective Value (Maximum Profit): $", model.objective())
for m in months:
    print(f"\nMonth {m}:")
    for c in colors:
        color = "Red" if c == 1 else "Black"
        print(f"{color} shirts produced: {model.x[m, c]()}")
        print(f"{color} shirts sold: {model.y[m, c]()}")
        print(f"{color} shirts in inventory: {model.e[m, c]()}")

print("\nExtra Production (beyond 500 shirts):")
for m in months:
    print(f"Month {m}: {model.z[m]()} shirts")


Objective Value (Maximum Profit): $ 40440.0

Month 1:
Red shirts produced: 250.0
Red shirts sold: 250.0
Red shirts in inventory: 0.0
Black shirts produced: 200.0
Black shirts sold: 200.0
Black shirts in inventory: 0.0

Month 2:
Red shirts produced: 90.0
Red shirts sold: 90.0
Red shirts in inventory: 0.0
Black shirts produced: 360.0
Black shirts sold: 160.0
Black shirts in inventory: 200.0

Month 3:
Red shirts produced: 250.0
Red shirts sold: 250.0
Red shirts in inventory: 0.0
Black shirts produced: 250.0
Black shirts sold: 450.0
Black shirts in inventory: 0.0

Month 4:
Red shirts produced: 350.0
Red shirts sold: 350.0
Red shirts in inventory: 0.0
Black shirts produced: 300.0
Black shirts sold: 300.0
Black shirts in inventory: 0.0

Month 5:
Red shirts produced: 320.0
Red shirts sold: 320.0
Red shirts in inventory: 0.0
Black shirts produced: 200.0
Black shirts sold: 200.0
Black shirts in inventory: 0.0

Month 6:
Red shirts produced: 380.0
Red shirts sold: 380.0
Red shirts in inventory: 0

In [ ]:
import pyomo.environ as pyo
from pyomo.environ import ConcreteModel, Var, NonNegativeReals, Objective, Constraint, SolverFactory

def build_and_solve_model():
    m = ConcreteModel()

    # Define Decision Variables
    production_vars = ["p1T", "p1C", "p2T", "p2C"]
    shipping_vars = [
        "x_1DC1T", "x_1DC2T", "x_1S1T",
        "x_2DC1T", "x_2DC2T", "x_2S2T",
        "x_DC1S1T", "x_DC1S2T", "x_DC2S1T", "x_DC2S2T",
        "x_1DC1C", "x_1DC2C", "x_1S1C",
        "x_2DC1C", "x_2DC2C", "x_2S2C",
        "x_DC1S1C", "x_DC1S2C", "x_DC2S1C", "x_DC2S2C"
    ]

    for var in production_vars + shipping_vars:
        setattr(m, var, Var(domain=NonNegativeReals))

    # Objective Function: all i got to do is  Minimize Production + Shipping Costs
    def objective_rule(m):
        cost_prod = 1200*m.p1T + 100*m.p1C + 1275*m.p2T + 95*m.p2C
        cost_ship_tables = (
            150*m.x_1S1T + 30*m.x_1DC1T + 70*m.x_1DC2T +
            50*m.x_2S2T + 35*m.x_2DC1T + 15*m.x_2DC2T +
            55*m.x_DC1S1T + 10*m.x_DC1S2T +
            25*m.x_DC2S1T + 15*m.x_DC2S2T
        )
        cost_ship_chairs = (
            18*m.x_1S1C + 5*m.x_1DC1C + 8*m.x_1DC2C +
            6*m.x_2S2C + 3*m.x_2DC1C + 2*m.x_2DC2C +
            9*m.x_DC1S1C + 10*m.x_DC1S2C +
            5*m.x_DC2S1C + 3*m.x_DC2S2C
        )
        return cost_prod + cost_ship_tables + cost_ship_chairs

    m.obj = Objective(rule=objective_rule, sense=pyo.minimize)

    #  Labor Constraints( chatgbt)
    m.plant1_labor = Constraint(expr=10*m.p1T + m.p1C <= 240)
    m.plant2_labor = Constraint(expr=10*m.p2T + m.p2C <= 150)

    #  Production to Shipping Constraints
    m.plant1_tables = Constraint(expr=m.x_1S1T + m.x_1DC1T + m.x_1DC2T <= m.p1T)
    m.plant1_chairs = Constraint(expr=m.x_1S1C + m.x_1DC1C + m.x_1DC2C <= m.p1C)
    m.plant2_tables = Constraint(expr=m.x_2S2T + m.x_2DC1T + m.x_2DC2T <= m.p2T)
    m.plant2_chairs = Constraint(expr=m.x_2S2C + m.x_2DC1C + m.x_2DC2C <= m.p2C)

    #   DC Flow Balance Constraints
    m.dc1_tables = Constraint(expr=m.x_1DC1T + m.x_2DC1T == m.x_DC1S1T + m.x_DC1S2T)
    m.dc1_chairs = Constraint(expr=m.x_1DC1C + m.x_2DC1C == m.x_DC1S1C + m.x_DC1S2C)
    m.dc2_tables = Constraint(expr=m.x_1DC2T + m.x_2DC2T == m.x_DC2S1T + m.x_DC2S2T)
    m.dc2_chairs = Constraint(expr=m.x_1DC2C + m.x_2DC2C == m.x_DC2S1C + m.x_DC2S2C)

    #   Demand Constraints at Stores
    m.store1_tables = Constraint(expr=m.x_1S1T + m.x_DC1S1T + m.x_DC2S1T == 8)
    m.store1_chairs = Constraint(expr=m.x_1S1C + m.x_DC1S1C + m.x_DC2S1C == 85)
    m.store2_tables = Constraint(expr=m.x_2S2T + m.x_DC1S2T + m.x_DC2S2T == 12)
    m.store2_chairs = Constraint(expr=m.x_2S2C + m.x_DC1S2C + m.x_DC2S2C == 60)

    #  DC1 and DC2 Space Constraints
    m.dc1_capacity = Constraint(expr=50*(m.x_DC1S1T + m.x_DC1S2T) + 6*(m.x_DC1S1C + m.x_DC1S2C) <= 300)
    m.dc2_capacity = Constraint(expr=42*(m.x_DC2S1T + m.x_DC2S2T) + 5*(m.x_DC2S1C + m.x_DC2S2C) <= 210)

    # Solve with CBC Solver
    solver = SolverFactory('cbc')
    results = solver.solve(m, tee=True)

    # Print solution
    print("===== Optimal Decision Variable Values =====")
    for var in production_vars + shipping_vars:
        value = pyo.value(getattr(m, var))
        if value is not None and abs(value) > 1e-6:
            print(f"{var} = {value:.2f}")

    print("\n===== Optimal Objective Value (Total Cost) =====")
    print(f"Z = {pyo.value(m.obj):.3f}")

if __name__ == "__main__":
    build_and_solve_model()


Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpb1y09tic.pyomo.lp -stat=1 -solve -solu /tmp/tmpb1y09tic.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 16 (0) rows, 24 (0) columns and 56 (0) elements
Statistics for presolved model


Problem has 16 rows, 24 columns (24 with objective) and 56 elements
Column breakdown:
24 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
4 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
4 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 4 of type L 0.0, 0 of type L 1.0, 
4 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Free 
Presolve 16 (0) rows, 24 (0) columns and 56 (0) elements
0  Obj 0 Primal inf 137.22591 (4)
21  Obj 41669.565